# 文档相似度

## 用途
   * 搜索引擎的类似文章推荐   
   * 购物网站的类似商品推荐
   * 点评网站/微博微信平台上的类似内容推荐

## 基于词袋模型的基本思路
   * 如果两个文档/两句话的用词越相似，他们的内容就应该越相似。因此，可以从词频入手，计算他们的相似程度
   * 文档向量化之后，相似度的考察就可以直接转化为计算空间中的距离问题
   * 缺陷： 不能考虑否定词的巨大作用，不能考虑词序的差异
   
### 在本质上，向量空间中文本相似度的计算和任何聚类方法所考虑的问题***没有区别***

## 余弦相似度
### 两个向量间的夹角能够很好的反映其相似度

   * 但夹角大小使用不便，因此用夹角的余弦值作为相似度衡量指标
   * 思考：为什么只考虑夹角，不考虑相对距离?
   
### 余弦值越接近1，夹角越接近0度，两个向量也就越相似
### 可以证明余弦值的计算公式可以直接扩展到n维空间
### 因此在由n维向量所构成的空间中，可以利用余弦值来计算文档的相似度

## 相似度计算：基本分析思路
### 语料分词、清理
  * 原始语料分词
  * 语料清理
### 语料向量化
  * 将语料转换为词频向量
  * 为了避免文章长度的差异，长度悬殊时可以考虑使用相对词频
### 计算相似度
  * 计算两个向量的余弦相似度，值越大表示越相似
### 仍然存在的问题
  * 高频词不一定具有文档代表性，导致相似度计算结果变差

## 相似度计算：基本分析思路
### 语料分词、清理
  * 原始语料分词
  * 语料清理
### 语料向量化
  * 将语料转换为基于关键词的词频向量
  * 为了避免文章长度的差异，长度悬殊时可以考虑使用相对词频
### 使用TF-IDF算法，找出两篇文章的关键词
  * 例如取前20个，或者前50个
### 计算相似度
  * 计算两个向量的余弦相似度，值越大表示越相似

### 当向量表示概率分布式，其他相似度测量方法比余弦相似度更好

## 词条相似度：word2vec

词袋模型不考虑词条之间的相关性，因此无法用于计算词条相似度。

分布式表达会考虑词条的上下文关联，因此能够提取出词条上下文中的相关性信息，而词条之间的相似度就可以直接利用此类信息加以计算。

目前主要使用gensim实现相应的算法。

gensim也提供了sklearn的API接口：sklearn_api.w2vmodel，可以在sklearn中直接使用。

设置word2vec模型
> class gensim.models.word2vec.Word2Vec(
>
> sentences = None : 类似list of list的格式，对于特别大的文本，尽量考虑流式处理
>
> size = 100 : 词条向量的维度，数据量充足时，300/500的效果会更好
>
> window = 5 : 上下文窗口大小
>
> workers = 3 : 同时运行的线程数，多核系统可明显加速计算
>
>其余细节参数设定：
>
>    min_count = 5 : 低频词过滤阈值，低于该词频的不纳入模型
>
>    max_vocab_size = None : 每1千万词条需要1G内存，必要时设定该参数以节约内存
>
>    sample=0.001 : 负例采样的比例设定
>
>    negative=5 : 一般为5-20，设为0时不进行负例采样
>
>    iter = 5 : 模型在语料库上的迭代次数，该参数将被取消
>
>与神经网络模型有关的参数设定：
>
>    seed=1, alpha=0.025, min_alpha=0.0001, sg=0, hs=0
>
>)

In [2]:
import pandas as pd
# 有的环境配置下read_table出错，因此改用read_csv
raw = pd.read_csv("金庸-射雕英雄传txt精校版.txt",
                  names = ['txt'], sep ='aaa', encoding ="GBK" ,engine='python')
# 章节判断用变量预处理
def m_head(tmpstr):
    return tmpstr[:1]

def m_mid(tmpstr):
    return tmpstr.find("回 ")

raw['head'] = raw.txt.apply(m_head)
raw['mid'] = raw.txt.apply(m_mid)
raw['len'] = raw.txt.apply(len)
# raw['chap'] = 0
raw.head(0)
# 章节判断
chapnum = 0
for i in range(len(raw)):
    if raw['head'][i] == "第" and raw['mid'][i] > 0 and raw['len'][i] < 30 :
        chapnum += 1
    if chapnum >= 40 and raw['txt'][i] == "附录一：成吉思汗家族" :
        chapnum = 0
    raw.loc[i, 'chap'] = chapnum
    
# 删除临时变量
del raw['head']
del raw['mid']
del raw['len']
raw.head(0)
#提取章节
rawgrp = raw.groupby('chap')
chapter = rawgrp.sum()##.agg(sum) # 只有字符串列的情况下，sum函数自动转为合并字符串
chapter = chapter[chapter.index != 0]

In [3]:
chapter.head()

,txt
chap,
1.0,第一回 风雪惊变钱塘江浩浩江水，日日夜夜无穷无休的从两浙西路临安府牛家村边绕过，东流入海。江...
2.0,第二回 江南七怪颜烈跨出房门，过道中一个中年士人拖着鞋皮，踢跶踢跶的直响，一路打着哈欠迎面过...
3.0,第三回 黄沙莽莽寺里僧众见焦木圆寂，尽皆悲哭。有的便为伤者包扎伤处，抬入客舍。忽听得巨钟下的...
4.0,第四回 黑风双煞完颜洪熙笑道：“好，再打他个痛快。”蒙古兵前哨报来：“王罕亲自前来迎接大金国...
5.0,第五回 弯弓射雕一行人下得山来，走不多时，忽听前面猛兽大吼声一阵阵传来。韩宝驹一提缰，胯下黄...


In [4]:
# 分词和预处理，生成list of list格式
import jieba

chapter['cut'] = chapter.txt.apply(jieba.lcut)
chapter.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.821 seconds.
Prefix dict has been built succesfully.


,txt,cut
chap,,
1.0,第一回 风雪惊变钱塘江浩浩江水，日日夜夜无穷无休的从两浙西路临安府牛家村边绕过，东流入海。江...,"[第一回, , 风雪, 惊变, 钱塘江, 浩浩, 江水, ，, 日日夜夜, 无穷, 无休,..."
2.0,第二回 江南七怪颜烈跨出房门，过道中一个中年士人拖着鞋皮，踢跶踢跶的直响，一路打着哈欠迎面过...,"[第二回, , 江南七怪, 颜烈, 跨出, 房门, ，, 过道, 中, 一个, 中年, 士..."
3.0,第三回 黄沙莽莽寺里僧众见焦木圆寂，尽皆悲哭。有的便为伤者包扎伤处，抬入客舍。忽听得巨钟下的...,"[第三回, , 黄沙, 莽莽, 寺里, 僧众, 见, 焦木, 圆寂, ，, 尽, 皆, 悲..."
4.0,第四回 黑风双煞完颜洪熙笑道：“好，再打他个痛快。”蒙古兵前哨报来：“王罕亲自前来迎接大金国...,"[第四回, , 黑风双, 煞, 完颜洪熙, 笑, 道, ：, “, 好, ，, 再, 打,..."
5.0,第五回 弯弓射雕一行人下得山来，走不多时，忽听前面猛兽大吼声一阵阵传来。韩宝驹一提缰，胯下黄...,"[第五回, , 弯弓, 射雕, 一行, 人下, 得, 山来, ，, 走, 不多时, ，, ..."


In [5]:
# 初始化word2vec模型和词表
from gensim.models.word2vec import Word2Vec

n_dim = 300 # 指定向量维度，大样本量时300~500较好

w2vmodel = Word2Vec(size = n_dim, min_count = 10)
w2vmodel.build_vocab(chapter.cut) # 生成词表
w2vmodel

***对word2vec模型进行训练***

>word2vecmodel.train(
>
> sentences : iterable of iterables格式，对于特别大量的文本，尽量考虑流式处理
>
>total_examples = None : 句子总数，int，可直接使用model.corpus_count指定
>
>total_words = None : 句中词条总数，int，该参数和total_examples至少要指定一个
>
>epochs = None : 模型迭代次数，需要指定
>
>其他带默认值的参数设定：
>
>   start_alpha=None, end_alpha=None, word_count=0, queue_factor=2,
>
>   report_delay=1.0, compute_loss=False, callbacks=()
>
>)

In [6]:
# 在评论训练集上建模（大数据集时可能会花费几分钟）
# 本例消耗内存较少
#time 
w2vmodel.train(chapter.cut, \
               total_examples = w2vmodel.corpus_count, epochs = 10)

(3409733, 5975050)

In [7]:
# 训练完毕的模型实质
print(w2vmodel.wv["郭靖"].shape)
w2vmodel.wv["郭靖"]

(300,)


array([-1.4949511 , -0.00842526, -0.2171623 ,  0.7543341 , -0.04970288,
       -0.24352519,  0.53647155, -0.5041277 ,  0.8033062 ,  0.19798397,
       -0.14323044,  0.04606513,  0.47363696,  0.01851266,  0.1761883 ,
       -0.14015183, -0.19939011,  0.10235862,  0.03061349, -0.5380554 ,
        0.49867526, -0.09631232,  0.6954292 , -0.10060424,  0.11374087,
        0.63281137,  0.12074257, -0.23394747,  0.47420624,  0.47054785,
        0.15025923, -0.9008722 , -0.05997993, -0.05178253, -0.5275885 ,
       -0.0306479 ,  0.23682885, -0.01469101,  0.26895013, -0.23592488,
        0.6329945 , -0.12705204, -0.48065042,  0.49208015,  0.49458185,
        0.28157103, -0.2111737 ,  0.10984179, -0.23676407, -0.12948093,
        0.17917465,  0.05942281,  0.39504817, -0.71047693,  0.44548586,
        0.397497  , -0.01337112, -0.55380344, -1.0973364 , -0.13230963,
       -0.6144392 , -0.14237542,  0.29108572,  0.35000265,  0.06871293,
       -0.04935078,  0.23051688,  0.58580273, -0.0274988 ,  0.79

w2v模型的保存和复用

> w2vmodel.save(存盘路径及文件名称)
> w2vmodel.load(存盘路径及文件名称)

词向量间的相似度
> w2vmodel.wv.most_similar(词条)

In [8]:
w2vmodel.wv.most_similar("郭靖")

[('黄蓉', 0.9139183163642883),
 ('欧阳克', 0.8371074199676514),
 ('穆念慈', 0.7574751377105713),
 ('程瑶迦', 0.7457101345062256),
 ('完颜康', 0.7375081181526184),
 ('柯镇恶', 0.7307557463645935),
 ('裘千仞', 0.7294843196868896),
 ('梅超风', 0.7269957065582275),
 ('杨康', 0.7257514595985413),
 ('穆易', 0.7191975116729736)]

In [9]:
w2vmodel.wv.most_similar("黄蓉", topn = 20)

[('郭靖', 0.9139183759689331),
 ('欧阳克', 0.8899412155151367),
 ('穆念慈', 0.8307197690010071),
 ('完颜康', 0.8280520439147949),
 ('程瑶迦', 0.8240326642990112),
 ('包惜弱', 0.8121179342269897),
 ('穆易', 0.7835402488708496),
 ('裘千仞', 0.7831634283065796),
 ('洪七公', 0.7824699282646179),
 ('陆冠英', 0.7788628339767456),
 ('周伯通', 0.7728621959686279),
 ('一灯', 0.7693881988525391),
 ('杨康', 0.7622434496879578),
 ('柯镇恶', 0.7571394443511963),
 ('那公子', 0.7453505992889404),
 ('欧阳锋', 0.741683840751648),
 ('华筝', 0.7287399768829346),
 ('鲁有脚', 0.7231466770172119),
 ('黄药师', 0.7229665517807007),
 ('梅超风', 0.7198961973190308)]

In [10]:
w2vmodel.wv.most_similar("黄蓉道")

[('郭靖道', 0.9571603536605835),
 ('傻姑道', 0.8834599852561951),
 ('朱聪道', 0.8736648559570312),
 ('杨康道', 0.8528420925140381),
 ('头道', 0.7910284996032715),
 ('怒道', 0.7819041013717651),
 ('郭靖摇', 0.7596696615219116),
 ('喜道', 0.748505711555481),
 ('那人道', 0.7314038276672363),
 ('黄蓉笑', 0.7236558198928833)]

In [11]:
# 寻找对应关系
w2vmodel.wv.most_similar(['郭靖', '小红马'], ['黄药师'], topn = 5)

[('奔', 0.8441349267959595),
 ('悄悄', 0.8150516152381897),
 ('过去', 0.8066216707229614),
 ('晕', 0.8034513592720032),
 ('推开', 0.8014492988586426)]

In [12]:
w2vmodel.wv.most_similar(positive=['郭靖', '黄蓉'], negative=['杨康'], topn=10)

[('欧阳克', 0.7425419092178345),
 ('洪七公', 0.6639356017112732),
 ('她', 0.6480711698532104),
 ('欧阳锋', 0.6412407755851746),
 ('梅超风', 0.6144818663597107),
 ('主人', 0.586811900138855),
 ('口中', 0.5859293937683105),
 ('周伯通', 0.5845122337341309),
 ('那公子', 0.5836194753646851),
 ('他', 0.5728038549423218)]

In [13]:
# 计算两个词的相似度/相关程度
print(w2vmodel.wv.similarity("郭靖", "黄蓉"))
print(w2vmodel.wv.similarity("郭靖", "杨康"))
print(w2vmodel.wv.similarity("郭靖", "杨铁心"))

0.9139184
0.72575146
0.66478753


In [14]:
# 寻找不合群的词
w2vmodel.wv.doesnt_match("小红马 黄药师 鲁有脚".split())

C:\Users\Administrator\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'小红马'

In [15]:
w2vmodel.wv.doesnt_match("杨铁心 黄药师 黄蓉 洪七公".split())

'杨铁心'

## 文档相似度
### 基于词袋模型计算
***sklearn实现***
>sklearn.metrics.pairwise.pairwise_distances(
>
>X : 用于计算距离的数组
>
>  \[n_samples_a, n_samples_a\] if metric == 'precomputed'
>
>   \[n_samples_a, n_features\] otherwise
>
>Y = None : 用于计算距离的第二数组，当metric != 'precomputed'时可用
>
>metric = 'euclidean' : 空间距离计算方式
>
>scikit-learn原生支持 : \['cityblock', 'cosine', 'euclidean', 
>
>        'l1', 'l2', 'manhattan'\]，可直接使用稀疏矩阵格式
>
>    来自scipy.spatial.distance : \['braycurtis', 'canberra', 
>
>    'chebyshev', 'correlation', 'dice', 'hamming', 'jaccard',
>
>        'kulsinski', 'mahalanobis', 'matching', 'minkowski',
>
>        'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener',
>
>        'sokalsneath', 'sqeuclidean', 'yule'\] 不支持稀疏矩阵格式
>
>n_jobs = 1 : 用于计算的线程数，为-1时，所有CPU内核都用于计算
>
>)

In [16]:
cleanchap = [ " ".join(m_cut(w)) for w in chapter.txt.iloc[:5]] 

from sklearn.feature_extraction.text import CountVectorizer

countvec = CountVectorizer() 

resmtx = countvec.fit_transform(cleanchap)
resmtx

NameError: name 'm_cut' is not defined

In [17]:
from sklearn.metrics.pairwise import pairwise_distances

pairwise_distances(resmtx, metric = 'cosine')

NameError: name 'resmtx' is not defined

In [ ]:
pairwise_distances(resmtx) # 默认值为euclidean

In [ ]:
# 使用TF-IDF矩阵进行相似度计算
pairwise_distances(tfidf[:5], metric = 'cosine')

***gensim实现***

***基于LDA计算余弦相似度***

需要使用的信息：

    拟合完毕的lda模型
    按照拟合模型时矩阵种类转换的需检索文本
        需检索的文本
        建模时使用的字典

In [ ]:
from gensim import similarities
simmtx = similarities.MatrixSimilarity(corpus)
simmtx

In [ ]:
# 检索和第1章内容最相似（所属主题相同）的章节
simmtx = similarities.MatrixSimilarity(corpus) # 使用的矩阵种类需要和拟合模型时相同
simmtx

In [ ]:
simmtx.index[:2]

In [ ]:
# 使用gensim的LDA拟合结果进行演示
query = chapter.txt[1] 
query_bow = dictionary.doc2bow(m_cut(query))

lda_vec = ldamodel[query_bow] # 转换为lda模型下的向量
sims = simmtx[lda_vec] # 进行矩阵内向量和所提供向量的余弦相似度查询
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims

### doc2vec
word2vec用来计算词条相似度非常合适。

较短的文档如果希望计算文本相似度，可以将各自内部的word2vec向量分别进行平均，用平均后的向量作为文本向量，从而用于计算相似度。

但是对于长文档，这种平均的方式显然过于粗糙。

doc2vec是word2vec的拓展，它可以直接获得sentences/paragraphs/documents的向量表达，从而可以进一步通过计算距离来得到sentences/paragraphs/documents之间的相似性。

模型概况

    分析目的：获得文档的一个固定长度的向量表达。
    数据：多个文档，以及它们的标签，一般可以用标题作为标签。 
    影响模型准确率的因素：语料的大小，文档的数量，越多越高；文档的相似性，越相似越好。

In [ ]:
import jieba 
import gensim
from gensim.models import doc2vec

def m_doc(doclist):
    reslist = []
    for i, doc in enumerate(doclist):
        reslist.append(doc2vec.TaggedDocument(jieba.lcut(doc), [i]))
    return reslist

corp = m_doc(chapter.txt)

In [ ]:
corp[:2]

In [ ]:
d2vmodel = gensim.models.Doc2Vec(vector_size = 300, 
                window = 20, min_count = 5)
d2vmodel.build_vocab(corp)

In [ ]:
d2vmodel.wv.vocab

In [ ]:
# 将新文本转换为相应维度空间下的向量
newvec = d2vmodel.infer_vector(jieba.lcut(chapter.txt[1]))

In [ ]:
d2vmodel.docvecs.most_similar([newvec], topn = 10)